# Going through the [MSFT documentation on the Python Library for Digital Twin](https://docs.microsoft.com/en-us/python/api/overview/azure/digitaltwins-core-readme-pre?view=azure-python&source=docs)

Here are the things I've noticed so far:
* MSFT is [aware of a typo in the code](https://github.com/Azure/azure-sdk-for-python/issues/14918).

In [ ]:
# DefaultAzureCredential supports different authentication mechanisms and determines the appropriate credential type based of the environment it is executing in.
# It attempts to use multiple credential types in an order until it finds a working credential.

# - AZURE_URL: The tenant ID in Azure Active Directory
url = os.getenv("AZURE_URL")

# DefaultAzureCredential expects the following three environment variables:
# - AZURE_TENANT_ID: The tenant ID in Azure Active Directory
# - AZURE_CLIENT_ID: The application (client) ID registered in the AAD tenant
# - AZURE_CLIENT_SECRET: The client secret for the registered application
credential = DefaultAzureCredential()
serviceClient = DigitalTwinsClient(url, credential)